# CSE 6363 Project for Model Agnostic Meta Learning (MAML)
Daylen Griffen
Austin Johnson
Bailey Nguyen

# Normal Training Code (Single Task)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as models

# Transform for the dataset
transform = transforms.Compose([transforms.Resize((128, 128)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load datasets (we'll use a small subset for demonstration)
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Filter to get only cats (3) and dogs (5)
train_data.targets = [target for target in train_data.targets if target in [3, 5]]
train_data.data = train_data.data[train_data.targets]

test_data.targets = [target for target in test_data.targets if target in [3, 5]]
test_data.data = test_data.data[test_data.targets]

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define model (using a pre-trained ResNet18)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Output layer for cat and dog

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (normal training)
def train_normal():
    model.train()
    for epoch in range(5):  # Train for 5 epochs
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader)}")

# Evaluate
def evaluate_normal():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")

# Train and evaluate normal model
train_normal()
evaluate_normal()


# MAML Training Code (Meta-Training)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from higher import innerloop_ctx
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as models

# Same transformation as before
transform = transforms.Compose([transforms.Resize((128, 128)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load datasets for two categories (Cats vs Dogs)
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_data.targets = [target for target in train_data.targets if target in [3, 5]]
train_data.data = train_data.data[train_data.targets]

test_data.targets = [target for target in test_data.targets if target in [3, 5]]
test_data.data = test_data.data[test_data.targets]

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the model (ResNet18)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Output layer for 2 classes: cat and dog

# Loss function
criterion = nn.CrossEntropyLoss()

# Meta-learning parameters
meta_optimizer = optim.Adam(model.parameters(), lr=0.001)
meta_batch_size = 4  # Number of tasks in each batch

# MAML inner loop (task-specific adaptation)
def inner_loop(model, task_data, criterion, learning_rate=0.01):
    task_model = model
    optimizer = optim.SGD(task_model.parameters(), lr=learning_rate)
    inputs, labels = task_data
    optimizer.zero_grad()
    outputs = task_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    return task_model, loss

# MAML meta-training loop
def train_maml():
    model.train()
    for epoch in range(5):  # Train for 5 epochs
        meta_loss = 0
        for i, (inputs, labels) in enumerate(train_loader):
            # Create task-specific data (for simplicity we use the same task)
            task_data = (inputs, labels)

            # Perform MAML inner loop (adaptation on the task)
            task_model = model
            task_model, loss = inner_loop(task_model, task_data, criterion)

            # Meta-objective (outer loop)
            meta_loss += loss

        # Update meta-model (outer loop)
        meta_optimizer.zero_grad()
        meta_loss.backward()
        meta_optimizer.step()

        print(f"Epoch [{epoch+1}/5], Meta Loss: {meta_loss.item()}")

# Evaluate MAML model
def evaluate_maml():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")

# Train and evaluate using MAML
train_maml()
evaluate_maml()


Comparison:
1. Normal Training:
The model learns only for the specific task of classifying cats vs dogs.
The training process uses a larger dataset, and the model learns specific features for this dataset.
We train the model for 5 epochs, with the optimizer gradually adjusting the weights based on the loss calculated from the entire dataset.  

2. MAML Training:
In MAML, the model is trained over multiple tasks (even though in this simple example we have just one task: cats vs dogs).
The model adapts quickly by fine-tuning its parameters for each task using a few gradient steps in the inner loop.
The outer loop updates the model's parameters to make it easier for the model to adapt to new tasks in the future.  

Results:
Without MAML: The model will train for 5 epochs and show an accuracy on the test set based on the single task (cats vs dogs).  

With MAML: The model will also train for 5 epochs but will have the ability to adapt quickly to new tasks with minimal data, as it learns a good initialization during meta-training.  

In practice, the performance difference between normal training and MAML becomes more evident when you introduce tasks with limited data. MAML is designed to excel in few-shot learning situations where the model needs to adapt quickly to new tasks with very few examples.